In [79]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [263]:
# %load_ext autoreload
%autoreload 2
from pyspark.sql import SQLContext
import boto3
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sk_model import ReviewClassifier, vectorizer
import util

In [82]:
f_path = "s3://chizzy/yelp_data/"
df_bus = pd.read_json(f_path+"business.json", lines=True)
df_user = pd.read_json(f_path+"user.json", lines=True)
df_rev = pd.read_json(f_path+"review.json", lines=True)

In [83]:
df_bus.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,"4855 E Warner Rd, Ste B9","{'AcceptsInsurance': True, 'ByAppointmentOnly'...",FYWN1wneV18bWNgQjJ2GNg,"[Dentists, General Dentistry, Health & Medical...",Ahwatukee,"{'Friday': '7:30-17:00', 'Tuesday': '7:30-17:0...",1,33.330690,-111.978599,Dental by Design,,85044,22,4.0,AZ
1,3101 Washington Rd,"{'BusinessParking': {'garage': False, 'street'...",He-G7vWjzVUysIKrfNbPUQ,"[Hair Stylists, Hair Salons, Men's Hair Salons...",McMurray,"{'Monday': '9:00-20:00', 'Tuesday': '9:00-20:0...",1,40.291685,-80.104900,Stephen Szabo Salon,,15317,11,3.0,PA
2,"6025 N 27th Ave, Ste 1",{},KQPW8lFf1y5BT2MxiSZ3QA,"[Departments of Motor Vehicles, Public Service...",Phoenix,{},1,33.524903,-112.115310,Western Motor Vehicle,,85017,18,1.5,AZ
3,"5000 Arizona Mills Cr, Ste 435","{'BusinessAcceptsCreditCards': True, 'Restaura...",8DShNS-LuFqpEWIp0HxijA,"[Sporting Goods, Shopping]",Tempe,"{'Monday': '10:00-21:00', 'Tuesday': '10:00-21...",0,33.383147,-111.964725,Sports Authority,,85282,9,3.0,AZ
4,581 Howe Ave,"{'Alcohol': 'full_bar', 'HasTV': True, 'NoiseL...",PfOCPjBrlQAnz__NXj9h_w,"[American (New), Nightlife, Bars, Sandwiches, ...",Cuyahoga Falls,"{'Monday': '11:00-1:00', 'Tuesday': '11:00-1:0...",1,41.119535,-81.475690,Brick House Tavern + Tap,,44221,116,3.5,OH


In [84]:
df_user.head()

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,4.67,0,0,0,0,0,0,0,0,1,...,0,[],0,"[cvVMmlU1ouS3I5fhutaryQ, nj6UZ8tdGo8YJ9lUMTVWN...",0,Johnny,8,0,oMy_rEb0UBEmMlu-zcxnoQ,2014-11-03
1,3.70,0,0,0,0,0,0,0,0,0,...,0,[],0,"[0njfJmB-7n84DlIgUByCNw, rFn3Xe3RqHxRSxWOU19Gp...",0,Chris,10,0,JJ-aSuM4pCFPdkfoZ34q0Q,2013-09-24
2,2.00,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Tiffy,1,0,uUzsFQn_6cXDh6rPNGbIFA,2017-03-02
3,4.67,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Mark,6,0,mBneaEEH5EMyxaVyqS-72A,2015-03-13
4,4.67,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Evelyn,3,0,W5mJGs-dcDWRGEhAzUYtoA,2016-09-08


In [85]:
df_rev.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw
1,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw
2,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw
3,CKC0-MOWMqoeWf6s-szl8g,0,2016-05-28,0,MV3CcKScW05u5LVfF6ok0g,5,Lester's is located in a beautiful neighborhoo...,0,bv2nCi5Qv5vroFiqKGopiw
4,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw


In [86]:
#Rename colume names 
df_bus.rename(columns={"name":"bus_name","review_count":"bus_rev_count"}, inplace=True)
df_user.rename(columns={"name":"user_name", "review_count":"user_rev_count"}, inplace=True)
df_rev.rename(columns={"cool":"cool_rev", "funny":"funny_rev", "stars":"stars_rev", "useful":"useful_rev"}, inplace=True)

In [87]:
print("Number of rows for business table: {}".format(len(df_bus)))
print("Number of rows for user table: {}".format(len(df_user)))
print("Number of rows for review table: {}".format(len(df_rev)))

Number of rows for business table: 174567
Number of rows for user table: 1326101
Number of rows for review table: 5261669


In [88]:
us_states = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS", \
            "KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY", \
            "NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]

#Extracting reviews for only US restaurants
df_bus_us = df_bus[df_bus.state.isin (us_states)]

In [89]:
print("Number of rows for business table for US states: {}".format(len(df_bus_us)))

Number of rows for business table for US states: 128302


In [90]:
df_rev_bus = pd.merge(df_rev, df_bus, how='inner', on=['business_id', 'business_id'])

In [91]:
df_rev_bus.head()

,business_id,cool_rev,date,funny_rev,review_id,stars_rev,text,useful_rev,user_id,address,...,hours,is_open,latitude,longitude,bus_name,neighborhood,postal_code,bus_rev_count,stars,state
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw,3895 Boulevard Saint-Laurent,...,"{'Monday': '8:00-0:30', 'Tuesday': '8:00-0:30'...",1,45.516373,-73.577537,Schwartz's,Plateau-Mont-Royal,H2W 1X9,1953,4.0,QC
1,0W4lkclzZThpx3V65bVgig,0,2011-09-28,0,FunI9om-aK5oMIIJmhMlZA,4,a must stop when you are in montreal!\n\nI was...,0,u0LXt3Uea_GidxRW1xcsfg,3895 Boulevard Saint-Laurent,...,"{'Monday': '8:00-0:30', 'Tuesday': '8:00-0:30'...",1,45.516373,-73.577537,Schwartz's,Plateau-Mont-Royal,H2W 1X9,1953,4.0,QC
2,0W4lkclzZThpx3V65bVgig,2,2016-02-22,1,ufI413zxZqGwIBIInnM5Rg,4,*Schwartz's has become a Montreal's famous lan...,4,xYciRtVZ1PW4IxSX4oJ1aw,3895 Boulevard Saint-Laurent,...,"{'Monday': '8:00-0:30', 'Tuesday': '8:00-0:30'...",1,45.516373,-73.577537,Schwartz's,Plateau-Mont-Royal,H2W 1X9,1953,4.0,QC
3,0W4lkclzZThpx3V65bVgig,0,2013-07-01,0,cDUhgPLuLR8hQDNgkGh_hw,4,"Boss: ""You want to go to a conference in Mont...",0,ayaxadJdy0ObFKaMlgfNVw,3895 Boulevard Saint-Laurent,...,"{'Monday': '8:00-0:30', 'Tuesday': '8:00-0:30'...",1,45.516373,-73.577537,Schwartz's,Plateau-Mont-Royal,H2W 1X9,1953,4.0,QC
4,0W4lkclzZThpx3V65bVgig,0,2015-12-22,0,gE2sCFSGf8bhZ07uYRjpnw,4,I went to Montreal a few weeks ago Schwartz wa...,0,aU4ba7OlYKOKJ9kpvbOcsg,3895 Boulevard Saint-Laurent,...,"{'Monday': '8:00-0:30', 'Tuesday': '8:00-0:30'...",1,45.516373,-73.577537,Schwartz's,Plateau-Mont-Royal,H2W 1X9,1953,4.0,QC


In [92]:
df_rev_bus_user = pd.merge(df_rev_bus, df_user, how='left', on=['user_id', 'user_id'])

In [93]:
print("Number of rows for business table for concated business, user and review: {}".format(len(df_rev_bus_user)))

Number of rows for business table for concated business, user and review: 5261669


In [94]:
df_rev_bus_user.head()

,business_id,cool_rev,date,funny_rev,review_id,stars_rev,text,useful_rev,user_id,address,...,compliment_writer,cool,elite,fans,friends,funny,user_name,user_rev_count,useful,yelping_since
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw,3895 Boulevard Saint-Laurent,...,0,0,[],0,[4Zpn3ObC2zexkIhor4LJIQ],0,Tim,6,0,2015-12-22
1,0W4lkclzZThpx3V65bVgig,0,2011-09-28,0,FunI9om-aK5oMIIJmhMlZA,4,a must stop when you are in montreal!\n\nI was...,0,u0LXt3Uea_GidxRW1xcsfg,3895 Boulevard Saint-Laurent,...,3,22,"[2012, 2014, 2016, 2017, 2015, 2013]",8,"[QSpRpbemKx5U4-NtUumUbw, G-6X-llgA_qAxGxocykHz...",7,Julia,359,79,2009-08-20
2,0W4lkclzZThpx3V65bVgig,2,2016-02-22,1,ufI413zxZqGwIBIInnM5Rg,4,*Schwartz's has become a Montreal's famous lan...,4,xYciRtVZ1PW4IxSX4oJ1aw,3895 Boulevard Saint-Laurent,...,8,0,"[2015, 2017, 2016]",30,"[UX0Lh2n7gOiH5PVsCo1psA, GM7NRYBXxl3ELf8y4Loq0...",14,Anh,177,39,2015-02-15
3,0W4lkclzZThpx3V65bVgig,0,2013-07-01,0,cDUhgPLuLR8hQDNgkGh_hw,4,"Boss: ""You want to go to a conference in Mont...",0,ayaxadJdy0ObFKaMlgfNVw,3895 Boulevard Saint-Laurent,...,0,0,[],1,"[_V28Y9fkcQgo3-F24DDxTA, ZCIvJM3fuFYpUFIG4OFzj...",0,Tuan,26,1,2012-02-03
4,0W4lkclzZThpx3V65bVgig,0,2015-12-22,0,gE2sCFSGf8bhZ07uYRjpnw,4,I went to Montreal a few weeks ago Schwartz wa...,0,aU4ba7OlYKOKJ9kpvbOcsg,3895 Boulevard Saint-Laurent,...,1,6,[2017],1,"[PhJhg24zecO2USi_blq6eg, uOQTLqNUtiuNWxaTma02E...",3,Kenneth,32,10,2015-08-15


In [95]:
df_rev_bus_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5261669 entries, 0 to 5261668
Data columns (total 44 columns):
business_id           object
cool_rev              int64
date                  datetime64[ns]
funny_rev             int64
review_id             object
stars_rev             int64
text                  object
useful_rev            int64
user_id               object
address               object
attributes            object
categories            object
city                  object
hours                 object
is_open               int64
latitude              float64
longitude             float64
bus_name              object
neighborhood          object
postal_code           object
bus_rev_count         int64
stars                 float64
state                 object
average_stars         float64
compliment_cool       int64
compliment_cute       int64
compliment_funny      int64
compliment_hot        int64
compliment_list       int64
compliment_more       int64
compliment_note  

In [100]:
# with open("s3://chizzy/yelp_data/df_rev_bus_user.pickle", "wb") as f:
#     pickle.dump(df_rev_bus_user, f)

In [101]:
# with open("s3://chizzy/yelp_data/df_rev_bus_user.pickle", "rb") as f:
#     df = pickle.load(f)

In [234]:
df = df_rev_bus_user[["text", "stars_rev"]][df_rev_bus_user["state"]=="WI"]
df = df[df.stars_rev.isin([1,5])]
df.head()

,text,stars_rev
13482,"Gorgeous, peaceful, relaxing, but still being ...",5
13483,"Free and much larger than I expected, this bot...",5
13484,Fantastic - so many things to see in the conse...,5
13485,"Being new(ish) to Madison, I'm still figuring ...",5
13486,Olbrich Gardens is currently hosting GLEEM thr...,5


In [238]:

corpus = df["text"].values

In [239]:
print("Number of words reviews in corpus: {}".format(len(corpus)))

Number of words reviews in corpus: 55157


In [240]:
corpus[1]

'Free and much larger than I expected, this botanical garden was a pleasant surprise.  There are a few corners that are particularly nice, like the Rose Tower and Temple. Meander throughout and try to get lost. Sit a spell on a bench and enjoy the scenery.'

In [242]:
# nltk.download('stopwords')
# nltk.download('wordnet')

In [245]:
sw = set(stopwords.words("english"))
tokenizer = RegexpTokenizer("[\w']+")
lemma = WordNetLemmatizer()

In [246]:
cleaned = util.clean_stem(corpus, tokenizer, lemma, sw)

In [247]:
cleaned[1]

'free much larger expected botanical garden pleasant surprise corner particularly nice like rose tower temple meander throughout try get lost sit spell bench enjoy scenery'

In [248]:
print("Number of words before cleaning, and removing stopwords: {}".format(len(corpus[3].split())))
print("Number of words after cleaning, and removing stopwords: {}".format(len(cleaned[3].split())))

Number of words before cleaning, and removing stopwords: 44
Number of words after cleaning, and removing stopwords: 27


In [249]:
X, terms = vectorizer(cleaned, 
                       max_df=0.95, 
                       min_df=2,
                       stop_words='english')

y = df["stars_rev"].values

In [252]:
X.shape

(55157, 25591)

In [254]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

### LogisticRegression Linear model

In [255]:
model = ReviewClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [264]:
#Estimating metrics
matrix, recall, precision, accuracy = model.metrics_eval(X_test, y_test)

In [265]:
#confusion matrix
matrix

array([[10607,   410],
       [  103,  2670]])

In [266]:
print("Recall: {}%".format(round(recall*100, 2)))
print("Precision: {}%".format(round(precision*100, 2)))
print("Accuracy: {}%".format(round(accuracy*100, 2)))

Recall: 99.04%
Precision: 96.28%
Accuracy: 96.28%


In [267]:
# terms = tfidf_vectorizer.get_feature_names()
pos_terms, neg_terms = model.feature_importance(terms,40)

In [268]:
pos_terms

[('great', 11.960211521661957),
 ('delicious', 9.187577212490654),
 ('amazing', 9.135813660049637),
 ('best', 8.912434062844092),
 ('love', 7.962244374993826),
 ('excellent', 7.307862323061538),
 ('friendly', 6.970466318137969),
 ('awesome', 6.7239345575292635),
 ('favorite', 6.136279910602764),
 ('perfect', 5.791576576595871),
 ('fantastic', 5.745196272848464),
 ('highly', 5.4588843522208315),
 ('wonderful', 5.431002748846797),
 ('definitely', 5.354307256314705),
 ('helpful', 5.126041743376046),
 ('madison', 4.7064839490700745),
 ('happy', 4.379742850373381),
 ('thank', 4.287158805223204),
 ('loved', 4.081699898230515),
 ('reasonable', 4.077810030558766),
 ('fresh', 4.053110102738035),
 ('good', 3.8944549177813657),
 ('nice', 3.8643716893526268),
 ('outstanding', 3.6394629615478507),
 ('easy', 3.49918230433817),
 ('beautiful', 3.479859848944658),
 ('knowledgeable', 3.416220353778652),
 ('notch', 3.335047760905703),
 ('professional', 3.208831196913691),
 ('fun', 3.18332812852468),
 ('s

In [269]:
neg_terms

[('worst', -9.526141818973988),
 ('horrible', -7.163240385017844),
 ('rude', -7.0853905240058825),
 ('terrible', -7.041071224771245),
 ('awful', -5.939581712110233),
 ('poor', -5.797786821087562),
 ('told', -5.741305200279022),
 ('disappointing', -5.502132314821465),
 ('bad', -5.0724553708810864),
 ('bland', -5.035374911634823),
 ('worse', -4.544470984152732),
 ('money', -4.489255396663718),
 ('mediocre', -4.488292665640955),
 ('asked', -4.409721545298253),
 ('said', -4.232688714387224),
 ('dirty', -4.200351660156713),
 ('ok', -4.1892770977417895),
 ('overpriced', -4.082149728163625),
 ('disgusting', -4.079115657940544),
 ('waste', -3.7647818658057766),
 ('slow', -3.7055020217852146),
 ('guess', -3.631998675468537),
 ('left', -3.597642690419848),
 ('manager', -3.5355576013812136),
 ('ordered', -3.5193700436211017),
 ('unprofessional', -3.4883325901644717),
 ('tasted', -3.4647017166127627),
 ('unfortunately', -3.4620890572325824),
 ('gross', -3.390857435809842),
 ('minute', -3.381231382